In [ ]:
#@title # 初始化（要先執行）
#@markdown 先下載必要套件
!rm -rf /content/sample_data
!apt install -y -qq aria2
!pip install -qq gigafile mega.py
!mkdir -p /content/DL
mega_count = 0
%cd /content/DL

In [ ]:
#@title # 👇 **下載**
import re
%cd /content/DL
link = "" #@param{type:"string"}
from_direct_link = False #@param{type:"boolean"}
from_torrent = False #@param{type:"boolean"}
from_gigafile = False #@param{type:"boolean"}
from_pixeldrain = False #@param{type:"boolean"}
from_mega = False #@param{type:"boolean"}

if from_torrent:
  !aria2c --summary-interval=10 --seed-time=0 --allow-overwrite=true "{link}"

if from_direct_link:
  !aria2c --summary-interval=10 -c -x 16 -k 1M -s 16 "{link}"

if from_gigafile:
  !gfile download "{link}"

if from_pixeldrain:
  url_check = re.match('https://pixeldrain.com/u/', link)
  if url_check:
    pd_id = link[url_check.span()[1]: len(link)]
    bypass = 'https://pd.cybar.xyz/' + pd_id
    !aria2c --summary-interval=10 -c -x 16 -k 1M -s 16 "{bypass}"
  else:
    print("URL格式錯誤！")
    print("正確格式應為`https://pixeldrain.com/u/XXXXXXXX`")

if from_mega:
  if mega_count == 0:
    mega_count = 1
    from mega import Mega
    mega = Mega()
    m = mega.login()
  m.download_url(link)

print("\nDone!")

In [ ]:
#@title # 👇 **上傳**
from google.colab import drive
import os, json

google_drive = False #@param{type:"boolean"}
google_drive_path = "ColabDownloads/" #@param {type:"string"}
#@markdown **資料夾位置記得加 "/"**

#@markdown ---
#@markdown 以下皆需填filename(path也行)

#@markdown 有錯誤時可能是檔名中有特殊符號
filename = "" #@param{type:"string"}
#@markdown ---
#@markdown **Pixeldrain上限 [20](https://pixeldrain.com/) GB**
Pixeldrain = False #@param{type:"boolean"}
#@markdown ---
Gigafile = False #@param{type:"boolean"}
#@markdown ---
#@markdown catbox上限 [200](https://catbox.moe/) MB
catbox = False #@param{type:"boolean"}

if google_drive:
  if not os.path.exists('/content/gdrive'):
    drive.mount('/content/gdrive')
  !rsync -avh --progress . "/content/gdrive/MyDrive/"{output_path}
  print("\nDone!")

if Pixeldrain:
  pdupload = !curl -T "{filename}" https://pixeldrain.com/api/file/
  res = json.loads(pdupload[0])
  # output.clear()
  print("下載網址： https://pixeldrain.com/u/", res.get('id'), sep='')

if Gigafile:
  !gfile -p -n 24 upload "{filename}"

if catbox:
  !curl -F "reqtype=fileupload" -F "fileToUpload=@{filename}" https://catbox.moe/user/api.php

In [ ]:
#@title # 👇 **移除下載檔案 (清理一下colab)**
!rm -rf /content/DL/*
%cd /content/DL
print("乾淨ㄌ")

---
# 以下小工具


In [ ]:
#@title # Colab終端機
#@markdown [colab-xterm](https://github.com/InfuseAI/colab-xterm)
!pip install colab-xterm
%load_ext colabxterm
%xterm

In [ ]:
#@title # Telegram 工具
#@markdown [tdl](https://github.com/iyear/tdl) [文件](https://docs.iyear.me/tdl/)
!curl -sSL https://docs.iyear.me/tdl/install.sh | sudo bash
# upload tdl
# unzip tdl.zip -d ~/.tdl

In [ ]:
#@title # Mega帳號產生器
#@markdown [py_mega_account_generator](https://github.com/qtchaos/py_mega_account_generator)
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get update
!apt-get install ./google-chrome-stable_current_amd64.deb
!rm google-chrome-stable_current_amd64.deb
!git clone https://github.com/qtchaos/py_mega_account_generator.git
%cd py_mega_account_generator
!pip install -r requirements.txt
%cd ..
# 第一次執行把這個複製並貼上`/opt/google/chrome/chrome`